# 🛠️ Scikit-Learn: Regresión Lineal en Producción

## 📚 Objetivos de Aprendizaje
En este notebook aprenderás:
- **Transición** de implementación manual a herramientas profesionales
- **Scikit-Learn** para regresión lineal y polinomial
- **Pipeline completo** de ML con preprocessing automático
- **Validación robusta** con cross-validation
- **Comparación** entre implementación manual vs sklearn

## 🎯 ¿Por qué Scikit-Learn?
**Has aprendido los fundamentos** implementando desde cero:
- ✅ Gradient descent
- ✅ Feature scaling  
- ✅ Feature engineering
- ✅ Evaluación de modelos

**Ahora es momento de usar herramientas profesionales:**
- 🚀 **Velocidad**: Implementaciones optimizadas en C/Cython
- 🛡️ **Robustez**: Manejo de edge cases y errores
- 🔧 **Funcionalidad**: Pipelines, cross-validation, métricas
- 📈 **Escalabilidad**: Datasets grandes y algoritmos avanzados

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score, validation_curve, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import make_regression, load_diabetes
import time
import warnings
warnings.filterwarnings('ignore')

# Configuración
plt.style.use('default')
np.set_printoptions(precision=2, suppress=True)
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Scikit-Learn y librerías importadas")
print(f"📦 Sklearn version: {__import__('sklearn').__version__}")
print("🚀 Listo para ML profesional")

## 1. 🔄 Comparación: Manual vs Scikit-Learn

### 1.1 Implementación Manual (Repaso)

In [ ]:
# Funciones de implementación manual (del notebook anterior)
def gradient_descent_manual(X, y, learning_rate=0.01, n_iterations=1000):
    """Implementación manual de gradient descent"""
    m, n = X.shape
    w = np.zeros(n)
    b = 0
    costs = []
    
    for i in range(n_iterations):
        # Predicciones
        y_pred = X @ w + b
        
        # Costo
        cost = np.mean((y_pred - y)**2) / 2
        costs.append(cost)
        
        # Gradientes
        dw = X.T @ (y_pred - y) / m
        db = np.mean(y_pred - y)
        
        # Actualizar parámetros
        w -= learning_rate * dw
        b -= learning_rate * db
    
    return w, b, costs

def preprocessing_manual(X):
    """Normalización manual"""
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    X_normalized = (X - mu) / sigma
    return X_normalized, mu, sigma

# Crear dataset sintético para comparación
X_synthetic, y_synthetic = make_regression(
    n_samples=100, n_features=5, noise=10, random_state=42
)

print("🔬 Dataset sintético creado:")
print(f"   • Samples: {X_synthetic.shape[0]}")
print(f"   • Features: {X_synthetic.shape[1]}")
print(f"   • Target range: [{y_synthetic.min():.1f}, {y_synthetic.max():.1f}]")

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X_synthetic, y_synthetic, test_size=0.2, random_state=42
)

print(f"\n📊 División de datos:")
print(f"   • Entrenamiento: {X_train.shape[0]} ejemplos")
print(f"   • Prueba: {X_test.shape[0]} ejemplos")

### 1.2 Comparación Directa de Performance

In [ ]:
# Comparar implementación manual vs sklearn
print("⚡ COMPARACIÓN DE PERFORMANCE")
print("=" * 40)

# 1. IMPLEMENTACIÓN MANUAL
print("\n🔧 Implementación Manual:")
start_time = time.time()

# Preprocessing manual
X_train_norm, mu, sigma = preprocessing_manual(X_train)
X_test_norm = (X_test - mu) / sigma

# Entrenar modelo manual
w_manual, b_manual, costs_manual = gradient_descent_manual(
    X_train_norm, y_train, learning_rate=0.1, n_iterations=1000
)

# Predicciones manuales
y_pred_manual = X_test_norm @ w_manual + b_manual

time_manual = time.time() - start_time
mse_manual = np.mean((y_test - y_pred_manual)**2)
r2_manual = 1 - np.sum((y_test - y_pred_manual)**2) / np.sum((y_test - np.mean(y_test))**2)

print(f"   Tiempo: {time_manual:.4f} segundos")
print(f"   MSE: {mse_manual:.2f}")
print(f"   R²: {r2_manual:.4f}")
print(f"   Costo final: {costs_manual[-1]:.4f}")

# 2. SCIKIT-LEARN (Linear Regression - Solución Analítica)
print("\n🚀 Scikit-Learn (LinearRegression):")
start_time = time.time()

# Crear pipeline con scaling automático
sklearn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Entrenar
sklearn_pipeline.fit(X_train, y_train)

# Predicciones
y_pred_sklearn = sklearn_pipeline.predict(X_test)

time_sklearn = time.time() - start_time
mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)
r2_sklearn = r2_score(y_test, y_pred_sklearn)

print(f"   Tiempo: {time_sklearn:.4f} segundos")
print(f"   MSE: {mse_sklearn:.2f}")
print(f"   R²: {r2_sklearn:.4f}")

# 3. SCIKIT-LEARN (SGDRegressor - Gradient Descent)
print("\n⚡ Scikit-Learn (SGDRegressor):")
start_time = time.time()

sgd_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', SGDRegressor(max_iter=1000, alpha=0.01, random_state=42))
])

sgd_pipeline.fit(X_train, y_train)
y_pred_sgd = sgd_pipeline.predict(X_test)

time_sgd = time.time() - start_time
mse_sgd = mean_squared_error(y_test, y_pred_sgd)
r2_sgd = r2_score(y_test, y_pred_sgd)

print(f"   Tiempo: {time_sgd:.4f} segundos")
print(f"   MSE: {mse_sgd:.2f}")
print(f"   R²: {r2_sgd:.4f}")

# RESUMEN COMPARATIVO
print("\n📊 RESUMEN COMPARATIVO:")
print("-" * 50)
print(f"{'Método':<20} {'Tiempo':<10} {'MSE':<8} {'R²':<8} {'Speedup':<8}")
print("-" * 50)
print(f"{'Manual':<20} {time_manual:.4f}s   {mse_manual:<8.2f} {r2_manual:<8.4f} {'1.0x':<8}")
print(f"{'LinearRegression':<20} {time_sklearn:.4f}s   {mse_sklearn:<8.2f} {r2_sklearn:<8.4f} {time_manual/time_sklearn:<8.1f}x")
print(f"{'SGDRegressor':<20} {time_sgd:.4f}s   {mse_sgd:<8.2f} {r2_sgd:<8.4f} {time_manual/time_sgd:<8.1f}x")

print("\n💡 Observaciones:")
print("   • LinearRegression: Solución analítica exacta, muy rápida")
print("   • SGDRegressor: Gradient descent optimizado, escalable")
print("   • Manual: Educativo pero más lento")
print("   • Sklearn maneja automáticamente edge cases y optimizaciones")

## 2. 🔧 Scikit-Learn Fundamentals

### 2.1 API Consistente de Sklearn

In [ ]:
# Demostración del API consistente de sklearn
print("🔧 API CONSISTENTE DE SCIKIT-LEARN")
print("=" * 40)

# Todos los estimadores siguen el mismo patrón:
# 1. fit(X, y) - Entrenar
# 2. predict(X) - Predecir
# 3. score(X, y) - Evaluar

# Crear diferentes modelos con el mismo API
modelos = {
    'Linear Regression': LinearRegression(),
    'SGD Regressor': SGDRegressor(max_iter=1000, random_state=42),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0)
}

# Preprocessing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"{'Modelo':<18} {'R² Train':<10} {'R² Test':<10} {'MSE Test':<10} {'Características':<20}")
print("-" * 75)

resultados_modelos = {}

for nombre, modelo in modelos.items():
    # 1. FIT - Entrenar
    modelo.fit(X_train_scaled, y_train)
    
    # 2. PREDICT - Predecir
    y_pred_train = modelo.predict(X_train_scaled)
    y_pred_test = modelo.predict(X_test_scaled)
    
    # 3. SCORE - Evaluar
    r2_train = modelo.score(X_train_scaled, y_train)
    r2_test = modelo.score(X_test_scaled, y_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    
    # Características específicas del modelo
    if hasattr(modelo, 'coef_'):
        n_features_used = np.sum(np.abs(modelo.coef_) > 1e-10)
        caracteristica = f"{n_features_used}/{len(modelo.coef_)} features"
    else:
        caracteristica = "N/A"
    
    resultados_modelos[nombre] = {
        'modelo': modelo,
        'r2_train': r2_train,
        'r2_test': r2_test,
        'mse_test': mse_test,
        'y_pred': y_pred_test
    }
    
    print(f"{nombre:<18} {r2_train:<10.4f} {r2_test:<10.4f} {mse_test:<10.2f} {caracteristica:<20}")

print("\n🎯 Beneficios del API consistente:")
print("   • Mismos métodos para todos los modelos")
print("   • Fácil intercambio entre algoritmos")
print("   • Pipelines y automatización simples")
print("   • Curva de aprendizaje reducida")

# Demostrar intercambiabilidad
print(f"\n🔄 Ejemplo de intercambiabilidad:")
mejor_modelo_nombre = min(resultados_modelos.keys(), 
                         key=lambda k: resultados_modelos[k]['mse_test'])
mejor_modelo = resultados_modelos[mejor_modelo_nombre]['modelo']

print(f"   Mejor modelo: {mejor_modelo_nombre}")
print(f"   MSE: {resultados_modelos[mejor_modelo_nombre]['mse_test']:.2f}")
print(f"   R² test: {resultados_modelos[mejor_modelo_nombre]['r2_test']:.4f}")

### 2.2 Pipelines: Automatización del Workflow

In [ ]:
# Demostración de Pipelines para automatizar el workflow
print("🔄 PIPELINES: AUTOMATIZACIÓN DEL WORKFLOW")
print("=" * 45)

# Problema común: Múltiples pasos de preprocessing
print("❌ Problema sin Pipeline:")
print("   1. scaler = StandardScaler()")
print("   2. X_train_scaled = scaler.fit_transform(X_train)")
print("   3. X_test_scaled = scaler.transform(X_test)")
print("   4. model = LinearRegression()")
print("   5. model.fit(X_train_scaled, y_train)")
print("   6. predictions = model.predict(X_test_scaled)")
print("   → Múltiples objetos, propenso a errores\n")

print("✅ Solución con Pipeline:")

# Pipeline simple
pipeline_simple = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Una sola línea para entrenar todo el pipeline
pipeline_simple.fit(X_train, y_train)
y_pred_pipeline = pipeline_simple.predict(X_test)

print("   pipeline.fit(X_train, y_train)")
print("   predictions = pipeline.predict(X_test)")
print("   → Un solo objeto, automático, sin errores\n")

# Pipeline complejo con feature engineering
pipeline_complejo = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('feature_selection', 'passthrough'),  # Placeholder
    ('regressor', Ridge(alpha=1.0))
])

pipeline_complejo.fit(X_train, y_train)
y_pred_complejo = pipeline_complejo.predict(X_test)

# Comparar resultados
r2_simple = r2_score(y_test, y_pred_pipeline)
r2_complejo = r2_score(y_test, y_pred_complejo)
mse_simple = mean_squared_error(y_test, y_pred_pipeline)
mse_complejo = mean_squared_error(y_test, y_pred_complejo)

print(f"📊 Comparación de Pipelines:")
print(f"{'Pipeline':<15} {'R²':<8} {'MSE':<8} {'Features':<10}")
print("-" * 45)
print(f"{'Simple':<15} {r2_simple:<8.4f} {mse_simple:<8.2f} {X_train.shape[1]:<10}")
print(f"{'Complejo':<15} {r2_complejo:<8.4f} {mse_complejo:<8.2f} {pipeline_complejo.named_steps['poly_features'].transform(X_train).shape[1]:<10}")

# Beneficios de pipelines
print(f"\n🎯 Beneficios de Pipelines:")
print(f"   ✅ Previene data leakage")
print(f"   ✅ Código más limpio y mantenible")
print(f"   ✅ Fácil de usar en cross-validation")
print(f"   ✅ Reproducibilidad garantizada")
print(f"   ✅ Deployment simplificado")

# Inspeccionar pasos del pipeline
print(f"\n🔍 Inspección del Pipeline Complejo:")
for step_name, step_estimator in pipeline_complejo.named_steps.items():
    print(f"   {step_name}: {type(step_estimator).__name__}")
    
# Acceder a componentes específicos
poly_features = pipeline_complejo.named_steps['poly_features']
scaler = pipeline_complejo.named_steps['scaler']
regressor = pipeline_complejo.named_steps['regressor']

print(f"\n📈 Detalles del modelo final:")
print(f"   Features polinomiales: Grado {poly_features.degree}")
print(f"   Normalizador: {scaler.mean_[:3]:.2f}... (primeros 3 valores)")
print(f"   Regularización Ridge: Alpha = {regressor.alpha}")

## 3. 🎯 Validación Robusta con Cross-Validation

In [ ]:
# Cross-validation robusta para evaluación de modelos
print("🎯 VALIDACIÓN ROBUSTA CON CROSS-VALIDATION")
print("=" * 50)

# Cargar dataset real para demostración más realista
diabetes_data = load_diabetes()
X_diabetes, y_diabetes = diabetes_data.data, diabetes_data.target

print(f"📊 Dataset Diabetes:")
print(f"   • Samples: {X_diabetes.shape[0]}")
print(f"   • Features: {X_diabetes.shape[1]} ({list(diabetes_data.feature_names)})")
print(f"   • Target: Progresión de diabetes (continuo)")
print(f"   • Rango target: [{y_diabetes.min():.1f}, {y_diabetes.max():.1f}]")

# Crear diferentes pipelines para comparar
pipelines = {
    'Linear': Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ]),
    
    'Polynomial (deg=2)': Pipeline([
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ]),
    
    'Ridge (α=1.0)': Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Ridge(alpha=1.0))
    ]),
    
    'Lasso (α=1.0)': Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Lasso(alpha=1.0))
    ]),
    
    'SGD': Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', SGDRegressor(max_iter=1000, random_state=42))
    ])
}

# Cross-validation para cada pipeline
cv_resultados = {}
cv_folds = 5

print(f"\n🔄 Cross-Validation ({cv_folds}-fold):")
print(f"{'Modelo':<20} {'CV Mean':<10} {'CV Std':<8} {'Min':<8} {'Max':<8}")
print("-" * 60)

for nombre, pipeline in pipelines.items():
    # Realizar cross-validation
    cv_scores = cross_val_score(pipeline, X_diabetes, y_diabetes, 
                               cv=cv_folds, scoring='r2')
    
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    cv_min = cv_scores.min()
    cv_max = cv_scores.max()
    
    cv_resultados[nombre] = {
        'scores': cv_scores,
        'mean': cv_mean,
        'std': cv_std,
        'pipeline': pipeline
    }
    
    print(f"{nombre:<20} {cv_mean:<10.4f} {cv_std:<8.4f} {cv_min:<8.4f} {cv_max:<8.4f}")

# Identificar mejor modelo
mejor_modelo_cv = max(cv_resultados.keys(), key=lambda k: cv_resultados[k]['mean'])
mejor_pipeline = cv_resultados[mejor_modelo_cv]['pipeline']

print(f"\n🏆 Mejor modelo: {mejor_modelo_cv}")
print(f"   CV Score: {cv_resultados[mejor_modelo_cv]['mean']:.4f} ± {cv_resultados[mejor_modelo_cv]['std']:.4f}")

# Visualizar resultados de CV
plt.figure(figsize=(12, 8))

# Box plot de scores de CV
plt.subplot(2, 2, 1)
scores_data = [cv_resultados[nombre]['scores'] for nombre in pipelines.keys()]
plt.boxplot(scores_data, labels=list(pipelines.keys()))
plt.title('Distribución de CV Scores')
plt.ylabel('R² Score')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Comparación de medias con barras de error
plt.subplot(2, 2, 2)
nombres = list(cv_resultados.keys())
medias = [cv_resultados[nombre]['mean'] for nombre in nombres]
stds = [cv_resultados[nombre]['std'] for nombre in nombres]

bars = plt.bar(range(len(nombres)), medias, yerr=stds, capsize=5, alpha=0.7)
plt.title('CV Scores con Intervalos de Confianza')
plt.ylabel('R² Score')
plt.xticks(range(len(nombres)), nombres, rotation=45)
plt.grid(True, alpha=0.3, axis='y')

# Resaltar el mejor modelo
mejor_idx = nombres.index(mejor_modelo_cv)
bars[mejor_idx].set_color('gold')
bars[mejor_idx].set_edgecolor('black')
bars[mejor_idx].set_linewidth(2)

# Learning curves para el mejor modelo
plt.subplot(2, 2, 3)
from sklearn.model_selection import learning_curve

train_sizes, train_scores, val_scores = learning_curve(
    mejor_pipeline, X_diabetes, y_diabetes, cv=5, 
    train_sizes=np.linspace(0.1, 1.0, 10), scoring='r2'
)

train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2, color='blue')
plt.plot(train_sizes, val_mean, 'o-', color='red', label='Validation')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.2, color='red')

plt.title(f'Learning Curves: {mejor_modelo_cv}')
plt.xlabel('Training Set Size')
plt.ylabel('R² Score')
plt.legend()
plt.grid(True, alpha=0.3)

# Validation curves para regularización (si aplica)
plt.subplot(2, 2, 4)
if 'Ridge' in mejor_modelo_cv or 'Lasso' in mejor_modelo_cv:
    # Crear pipeline temporal para validation curve
    temp_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Ridge() if 'Ridge' in mejor_modelo_cv else Lasso())
    ])
    
    param_range = np.logspace(-3, 2, 10)
    train_scores_val, test_scores_val = validation_curve(
        temp_pipeline, X_diabetes, y_diabetes, 
        param_name='regressor__alpha', param_range=param_range,
        cv=5, scoring='r2'
    )
    
    train_mean_val = train_scores_val.mean(axis=1)
    test_mean_val = test_scores_val.mean(axis=1)
    
    plt.semilogx(param_range, train_mean_val, 'o-', color='blue', label='Training')
    plt.semilogx(param_range, test_mean_val, 'o-', color='red', label='Validation')
    plt.title('Validation Curve (Alpha)')
    plt.xlabel('Alpha (Regularization)')
    plt.ylabel('R² Score')
    plt.legend()
    plt.grid(True, alpha=0.3)
else:
    plt.text(0.5, 0.5, 'Validation Curve\nno aplicable\npara este modelo', 
             ha='center', va='center', transform=plt.gca().transAxes, 
             fontsize=12, bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.5))
    plt.title('Validation Curve')

plt.tight_layout()
plt.show()

print(f"\n💡 Insights de Cross-Validation:")
print(f"   • CV proporciona estimación robusta del rendimiento")
print(f"   • Desviación estándar indica estabilidad del modelo")
print(f"   • Learning curves muestran overfitting/underfitting")
print(f"   • Validation curves ayudan a optimizar hiperparámetros")

## 4. 🔍 Grid Search: Optimización Automática de Hiperparámetros

In [ ]:
# Grid Search para optimización automática de hiperparámetros
print("🔍 GRID SEARCH: OPTIMIZACIÓN DE HIPERPARÁMETROS")
print("=" * 55)

# Definir pipeline con múltiples pasos a optimizar
pipeline_optimizable = Pipeline([
    ('poly', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    ('regressor', Ridge())
])

# Definir grid de parámetros a buscar
param_grid = {
    'poly__degree': [1, 2, 3],  # Grado de características polinomiales
    'poly__include_bias': [True, False],  # Incluir término bias
    'regressor__alpha': [0.1, 1.0, 10.0, 100.0],  # Regularización Ridge
}

print(f"📋 Configuración de Grid Search:")
print(f"   • Pipeline: PolynomialFeatures → StandardScaler → Ridge")
print(f"   • Parámetros a optimizar:")
for param, valores in param_grid.items():
    print(f"     - {param}: {valores}")

total_combinaciones = np.prod([len(valores) for valores in param_grid.values()])
print(f"   • Total combinaciones: {total_combinaciones}")
print(f"   • CV folds: 5")
print(f"   • Total entrenamientos: {total_combinaciones * 5}")

# Ejecutar Grid Search
print(f"\n⚙️ Ejecutando Grid Search...")
start_time = time.time()

grid_search = GridSearchCV(
    pipeline_optimizable, 
    param_grid, 
    cv=5, 
    scoring='r2', 
    n_jobs=-1,  # Usar todos los cores disponibles
    verbose=1   # Mostrar progreso
)

grid_search.fit(X_diabetes, y_diabetes)
grid_time = time.time() - start_time

print(f"\n✅ Grid Search completado en {grid_time:.2f} segundos")

# Resultados del mejor modelo
print(f"\n🏆 MEJORES RESULTADOS:")
print(f"   • Mejor score: {grid_search.best_score_:.4f}")
print(f"   • Mejores parámetros:")
for param, valor in grid_search.best_params_.items():
    print(f"     - {param}: {valor}")

# Entrenar modelo final y evaluar
mejor_modelo_grid = grid_search.best_estimator_

# División train/test para evaluación final
X_train_diab, X_test_diab, y_train_diab, y_test_diab = train_test_split(
    X_diabetes, y_diabetes, test_size=0.2, random_state=42
)

mejor_modelo_grid.fit(X_train_diab, y_train_diab)
y_pred_final = mejor_modelo_grid.predict(X_test_diab)

# Métricas finales
r2_final = r2_score(y_test_diab, y_pred_final)
mse_final = mean_squared_error(y_test_diab, y_pred_final)
mae_final = mean_absolute_error(y_test_diab, y_pred_final)

print(f"\n📊 Evaluación en test set:")
print(f"   • R²: {r2_final:.4f}")
print(f"   • MSE: {mse_final:.2f}")
print(f"   • MAE: {mae_final:.2f}")

# Analizar top 10 combinaciones
resultados_df = pd.DataFrame(grid_search.cv_results_)
top_10 = resultados_df.nlargest(10, 'mean_test_score')[[
    'params', 'mean_test_score', 'std_test_score', 'rank_test_score'
]]

print(f"\n📈 Top 10 combinaciones:")
print(f"{'Rank':<5} {'Score':<8} {'Std':<8} {'Parámetros':<50}")
print("-" * 75)

for idx, row in top_10.iterrows():
    params_str = str(row['params'])[:50] + "..." if len(str(row['params'])) > 50 else str(row['params'])
    print(f"{int(row['rank_test_score']):<5} {row['mean_test_score']:<8.4f} {row['std_test_score']:<8.4f} {params_str:<50}")

# Visualizar resultados del grid search
plt.figure(figsize=(15, 10))

# Heatmap de results para degree vs alpha
plt.subplot(2, 3, 1)
pivot_data = resultados_df.pivot_table(
    values='mean_test_score', 
    index='param_poly__degree', 
    columns='param_regressor__alpha',
    aggfunc='mean'
)
plt.imshow(pivot_data.values, cmap='viridis', aspect='auto')
plt.colorbar()
plt.title('Heatmap: Degree vs Alpha')
plt.xlabel('Alpha (log scale)')
plt.ylabel('Polynomial Degree')
plt.xticks(range(len(pivot_data.columns)), [f'{alpha:.1f}' for alpha in pivot_data.columns])
plt.yticks(range(len(pivot_data.index)), pivot_data.index)

# Distribución de scores
plt.subplot(2, 3, 2)
plt.hist(resultados_df['mean_test_score'], bins=15, alpha=0.7, edgecolor='black')
plt.axvline(grid_search.best_score_, color='red', linestyle='--', 
           label=f'Best: {grid_search.best_score_:.4f}')
plt.title('Distribución de Scores')
plt.xlabel('Mean CV Score')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)

# Score vs polynomial degree
plt.subplot(2, 3, 3)
degree_scores = resultados_df.groupby('param_poly__degree')['mean_test_score'].agg(['mean', 'std'])
plt.errorbar(degree_scores.index, degree_scores['mean'], 
            yerr=degree_scores['std'], fmt='o-', capsize=5)
plt.title('Score vs Polynomial Degree')
plt.xlabel('Polynomial Degree')
plt.ylabel('Mean CV Score')
plt.grid(True, alpha=0.3)

# Score vs alpha
plt.subplot(2, 3, 4)
alpha_scores = resultados_df.groupby('param_regressor__alpha')['mean_test_score'].agg(['mean', 'std'])
plt.errorbar(alpha_scores.index, alpha_scores['mean'], 
            yerr=alpha_scores['std'], fmt='o-', capsize=5)
plt.xscale('log')
plt.title('Score vs Regularization (Alpha)')
plt.xlabel('Alpha')
plt.ylabel('Mean CV Score')
plt.grid(True, alpha=0.3)

# Predicciones vs realidad (test set)
plt.subplot(2, 3, 5)
plt.scatter(y_test_diab, y_pred_final, alpha=0.7)
plt.plot([y_test_diab.min(), y_test_diab.max()], 
         [y_test_diab.min(), y_test_diab.max()], 'r--', linewidth=2)
plt.title(f'Predictions vs Reality\nR² = {r2_final:.4f}')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.grid(True, alpha=0.3)

# Residuos
plt.subplot(2, 3, 6)
residuos = y_test_diab - y_pred_final
plt.scatter(y_pred_final, residuos, alpha=0.7)
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.title('Residuals Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 Conclusiones de Grid Search:")
print(f"   • Automatiza búsqueda exhaustiva de hiperparámetros")
print(f"   • Cross-validation previene overfitting en selección")
print(f"   • Paralelización acelera el proceso significativamente")
print(f"   • Visualización ayuda a entender impacto de parámetros")
print(f"   • Modelo final tiene rendimiento optimizado")

## 5. 📊 Comparación Final: Manual vs Sklearn

In [ ]:
# Comparación final exhaustiva entre implementación manual y sklearn
print("📊 COMPARACIÓN FINAL: MANUAL VS SKLEARN")
print("=" * 45)

# Crear dataset de benchmark
X_benchmark, y_benchmark = make_regression(
    n_samples=1000, n_features=10, noise=20, random_state=42
)

X_train_bench, X_test_bench, y_train_bench, y_test_bench = train_test_split(
    X_benchmark, y_benchmark, test_size=0.2, random_state=42
)

print(f"📈 Dataset de benchmark:")
print(f"   • Training: {X_train_bench.shape[0]} samples, {X_train_bench.shape[1]} features")
print(f"   • Test: {X_test_bench.shape[0]} samples")

# Funciones mejoradas para implementación manual
def implementacion_manual_completa(X_train, X_test, y_train, y_test):
    """Implementación manual completa con mejores prácticas"""
    start_time = time.time()
    
    # 1. Preprocessing
    mu = np.mean(X_train, axis=0)
    sigma = np.std(X_train, axis=0)
    sigma[sigma == 0] = 1  # Evitar división por cero
    
    X_train_scaled = (X_train - mu) / sigma
    X_test_scaled = (X_test - mu) / sigma
    
    # 2. Gradient descent mejorado
    m, n = X_train_scaled.shape
    w = np.random.normal(0, 0.01, n)  # Inicialización aleatoria pequeña
    b = 0
    
    learning_rate = 0.01
    n_iterations = 1000
    tolerance = 1e-6
    
    costs = []
    prev_cost = float('inf')
    
    for i in range(n_iterations):
        # Forward pass
        y_pred = X_train_scaled @ w + b
        
        # Costo
        cost = np.mean((y_pred - y_train)**2) / 2
        costs.append(cost)
        
        # Early stopping
        if abs(prev_cost - cost) < tolerance:
            break
        prev_cost = cost
        
        # Gradientes
        dw = X_train_scaled.T @ (y_pred - y_train) / m
        db = np.mean(y_pred - y_train)
        
        # Update
        w -= learning_rate * dw
        b -= learning_rate * db
    
    # 3. Predicciones
    y_pred_train = X_train_scaled @ w + b
    y_pred_test = X_test_scaled @ w + b
    
    training_time = time.time() - start_time
    
    return {
        'y_pred_train': y_pred_train,
        'y_pred_test': y_pred_test,
        'training_time': training_time,
        'n_iterations': i + 1,
        'final_cost': costs[-1],
        'costs': costs,
        'weights': w,
        'bias': b
    }

def implementacion_sklearn_completa(X_train, X_test, y_train, y_test):
    """Implementación sklearn completa con mejores prácticas"""
    start_time = time.time()
    
    # Pipeline optimizado
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ])
    
    # Entrenar
    pipeline.fit(X_train, y_train)
    
    # Predicciones
    y_pred_train = pipeline.predict(X_train)
    y_pred_test = pipeline.predict(X_test)
    
    training_time = time.time() - start_time
    
    return {
        'y_pred_train': y_pred_train,
        'y_pred_test': y_pred_test,
        'training_time': training_time,
        'pipeline': pipeline,
        'weights': pipeline.named_steps['regressor'].coef_,
        'bias': pipeline.named_steps['regressor'].intercept_
    }

# Ejecutar ambas implementaciones
print(f"\n⚙️ Ejecutando comparación...")

resultado_manual = implementacion_manual_completa(
    X_train_bench, X_test_bench, y_train_bench, y_test_bench
)

resultado_sklearn = implementacion_sklearn_completa(
    X_train_bench, X_test_bench, y_train_bench, y_test_bench
)

# Calcular métricas
def calcular_metricas(y_true, y_pred, nombre):
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return {'r2': r2, 'mse': mse, 'mae': mae, 'nombre': nombre}

metricas_manual_train = calcular_metricas(y_train_bench, resultado_manual['y_pred_train'], 'Manual Train')
metricas_manual_test = calcular_metricas(y_test_bench, resultado_manual['y_pred_test'], 'Manual Test')
metricas_sklearn_train = calcular_metricas(y_train_bench, resultado_sklearn['y_pred_train'], 'Sklearn Train')
metricas_sklearn_test = calcular_metricas(y_test_bench, resultado_sklearn['y_pred_test'], 'Sklearn Test')

# Mostrar comparación detallada
print(f"\n📊 RESULTADOS DETALLADOS:")
print(f"\n{'Métrica':<15} {'Manual Train':<12} {'Manual Test':<12} {'Sklearn Train':<13} {'Sklearn Test':<13}")
print("-" * 70)
print(f"{'R²':<15} {metricas_manual_train['r2']:<12.4f} {metricas_manual_test['r2']:<12.4f} {metricas_sklearn_train['r2']:<13.4f} {metricas_sklearn_test['r2']:<13.4f}")
print(f"{'MSE':<15} {metricas_manual_train['mse']:<12.2f} {metricas_manual_test['mse']:<12.2f} {metricas_sklearn_train['mse']:<13.2f} {metricas_sklearn_test['mse']:<13.2f}")
print(f"{'MAE':<15} {metricas_manual_train['mae']:<12.2f} {metricas_manual_test['mae']:<12.2f} {metricas_sklearn_train['mae']:<13.2f} {metricas_sklearn_test['mae']:<13.2f}")

print(f"\n⏱️  COMPARACIÓN DE TIEMPOS:")
print(f"   Manual: {resultado_manual['training_time']:.4f} segundos ({resultado_manual['n_iterations']} iteraciones)")
print(f"   Sklearn: {resultado_sklearn['training_time']:.4f} segundos (solución analítica)")
print(f"   Speedup: {resultado_manual['training_time'] / resultado_sklearn['training_time']:.1f}x más rápido sklearn")

# Comparar parámetros aprendidos
diferencia_weights = np.mean(np.abs(resultado_manual['weights'] - resultado_sklearn['weights']))
diferencia_bias = abs(resultado_manual['bias'] - resultado_sklearn['bias'])

print(f"\n🔍 CONVERGENCIA DE PARÁMETROS:")
print(f"   Diferencia promedio en pesos: {diferencia_weights:.6f}")
print(f"   Diferencia en bias: {diferencia_bias:.6f}")
print(f"   ✅ Convergencia exitosa" if diferencia_weights < 0.01 else "❌ Diferencias significativas")

# Visualización final comparativa
plt.figure(figsize=(16, 12))

# Convergencia del costo (manual)
plt.subplot(2, 3, 1)
plt.plot(resultado_manual['costs'])
plt.title(f'Convergencia Manual\n({resultado_manual["n_iterations"]} iteraciones)')
plt.xlabel('Iteraciones')
plt.ylabel('Cost')
plt.yscale('log')
plt.grid(True, alpha=0.3)

# Comparación de predicciones en test
plt.subplot(2, 3, 2)
plt.scatter(y_test_bench, resultado_manual['y_pred_test'], alpha=0.6, label='Manual', s=20)
plt.scatter(y_test_bench, resultado_sklearn['y_pred_test'], alpha=0.6, label='Sklearn', s=20)
plt.plot([y_test_bench.min(), y_test_bench.max()], 
         [y_test_bench.min(), y_test_bench.max()], 'r--', linewidth=2)
plt.title('Predicciones vs Realidad')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.legend()
plt.grid(True, alpha=0.3)

# Residuos comparativos
plt.subplot(2, 3, 3)
residuos_manual = y_test_bench - resultado_manual['y_pred_test']
residuos_sklearn = y_test_bench - resultado_sklearn['y_pred_test']
plt.hist(residuos_manual, bins=15, alpha=0.7, label='Manual', density=True)
plt.hist(residuos_sklearn, bins=15, alpha=0.7, label='Sklearn', density=True)
plt.axvline(x=0, color='red', linestyle='--', linewidth=2)
plt.title('Distribución de Residuos')
plt.xlabel('Residuos')
plt.ylabel('Densidad')
plt.legend()
plt.grid(True, alpha=0.3)

# Comparación de pesos
plt.subplot(2, 3, 4)
indices = range(len(resultado_manual['weights']))
plt.bar(np.array(indices) - 0.2, resultado_manual['weights'], width=0.4, alpha=0.7, label='Manual')
plt.bar(np.array(indices) + 0.2, resultado_sklearn['weights'], width=0.4, alpha=0.7, label='Sklearn')
plt.title('Comparación de Pesos')
plt.xlabel('Feature Index')
plt.ylabel('Peso')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

# Métricas comparativas
plt.subplot(2, 3, 5)
metricas = ['R² Train', 'R² Test', 'MSE Train', 'MSE Test']
valores_manual = [metricas_manual_train['r2'], metricas_manual_test['r2'], 
                 metricas_manual_train['mse']/1000, metricas_manual_test['mse']/1000]  # MSE/1000 para escala
valores_sklearn = [metricas_sklearn_train['r2'], metricas_sklearn_test['r2'],
                  metricas_sklearn_train['mse']/1000, metricas_sklearn_test['mse']/1000]

x_pos = np.arange(len(metricas))
plt.bar(x_pos - 0.2, valores_manual, width=0.4, alpha=0.7, label='Manual')
plt.bar(x_pos + 0.2, valores_sklearn, width=0.4, alpha=0.7, label='Sklearn')
plt.title('Métricas Comparativas')
plt.ylabel('Valor (MSE÷1000)')
plt.xticks(x_pos, metricas, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

# Tiempo de entrenamiento
plt.subplot(2, 3, 6)
tiempos = [resultado_manual['training_time'], resultado_sklearn['training_time']]
nombres = ['Manual\n(Gradient Descent)', 'Sklearn\n(Analytical)']
colores = ['lightcoral', 'lightgreen']
bars = plt.bar(nombres, tiempos, color=colores, alpha=0.7, edgecolor='black')
plt.title('Tiempo de Entrenamiento')
plt.ylabel('Segundos')
plt.grid(True, alpha=0.3, axis='y')

# Añadir valores en las barras
for bar, tiempo in zip(bars, tiempos):
    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.001,
             f'{tiempo:.4f}s', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Conclusiones finales
print(f"\n🎯 CONCLUSIONES FINALES:")
print(f"""
📚 VALOR EDUCATIVO DE IMPLEMENTACIÓN MANUAL:
   ✅ Comprensión profunda de algoritmos
   ✅ Entendimiento de gradient descent
   ✅ Insight sobre convergencia y optimización
   ✅ Base sólida para algoritmos avanzados

🚀 VENTAJAS DE SCIKIT-LEARN EN PRODUCCIÓN:
   ✅ Velocidad: Hasta {resultado_manual['training_time'] / resultado_sklearn['training_time']:.0f}x más rápido
   ✅ Robustez: Manejo automático de edge cases
   ✅ Precisión numérica: Algoritmos optimizados
   ✅ Ecosistema: Pipelines, CV, métricas integradas
   ✅ Mantenibilidad: Código más limpio y estándar
   ✅ Escalabilidad: Optimizado para datasets grandes

🎓 RECOMENDACIÓN:
   • Aprende los fundamentos implementando manualmente
   • Usa scikit-learn para proyectos reales
   • Combina ambos enfoques para máximo aprendizaje
""")

print(f"\n🏁 Has completado exitosamente el journey de Machine Learning:")
print(f"   Manual → Optimizado → Producción")
print(f"   ¡Ahora tienes las herramientas para proyectos ML profesionales!")

## 📚 Resumen Final del Journey Completo

### ✅ Journey Completado: De Manual a Profesional

#### 🗺️ **El Camino Recorrido**:
1. **Fundamentos Python/NumPy** → Herramientas básicas
2. **Regresión Lineal Manual** → Entendimiento profundo
3. **Múltiples Variables** → Escalabilidad de conceptos
4. **Feature Scaling** → Optimización de convergencia
5. **Feature Engineering** → Captura de patrones complejos
6. **Scikit-Learn** → Herramientas profesionales

#### 🎯 **Competencias Desarrolladas**:

**Nivel Fundamental** ✅:
- Gradient descent desde cero
- Funciones de costo y optimización
- Feature scaling y normalización
- Evaluación de modelos (R², MSE, MAE)

**Nivel Intermedio** ✅:
- Regresión multivariable
- Feature engineering y polinomial
- Overfitting vs underfitting
- Train/validation/test splits

**Nivel Profesional** ✅:
- Scikit-learn pipelines
- Cross-validation robusta
- Grid search automático
- Workflow completo de ML

### 🚀 **Próximos Pasos Sugeridos**:

#### **Profundizar en ML**:
- **Clasificación**: Regresión logística, SVM
- **Regularización**: Ridge, Lasso, Elastic Net
- **Ensemble Methods**: Random Forest, Gradient Boosting
- **Deep Learning**: Redes neuronales con TensorFlow/PyTorch

#### **Proyectos Reales**:
- Competencias en Kaggle
- Análisis de datasets reales
- Deployment de modelos (Flask, FastAPI)
- MLOps y automatización

#### **Especialización**:
- **Computer Vision**: CNNs, detección de objetos
- **NLP**: Transformers, BERT, GPT
- **Time Series**: ARIMA, LSTM, Prophet
- **Reinforcement Learning**: Q-learning, Policy Gradients

### 💼 **Para Aplicaciones Profesionales**:

#### **Usa Siempre**:
- ✅ **Scikit-learn** para modelos clásicos de ML
- ✅ **Pandas** para manipulación de datos
- ✅ **Cross-validation** para validación robusta
- ✅ **Pipelines** para workflows reproducibles
- ✅ **Grid/Random Search** para optimización

#### **Best Practices Profesionales**:
1. **Exploración de datos** antes de modelar
2. **División apropiada** de train/validation/test
3. **Feature engineering** basado en dominio
4. **Validación cruzada** para selección de modelo
5. **Monitoreo de rendimiento** en producción

### 🎓 **Lo que te Diferencia Ahora**:
- **Entiendes los fundamentos** (no solo usas librerías)
- **Puedes debuggear** algoritmos cuando fallan
- **Implementas soluciones** desde cero cuando es necesario
- **Optimizas** hyperparámetros sistemáticamente
- **Evalúas modelos** con rigor estadístico

### 🏆 **¡Felicitaciones!**

Has completado un journey comprehensivo de Machine Learning que te lleva desde los **fundamentos matemáticos** hasta las **herramientas profesionales**. 

**Ahora tienes:**
- 🧠 **Comprensión profunda** de cómo funcionan los algoritmos
- 🛠️ **Herramientas profesionales** para proyectos reales
- 📊 **Metodología rigurosa** para evaluación de modelos
- 🚀 **Base sólida** para aprender algoritmos avanzados

**¡Estás listo para hacer Machine Learning de nivel profesional!** 🎉